# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import gmaps
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazovsky,67.4667,78.7000,65.01,52,88,14.34,RU,1722764684
1,1,mount pearl,47.5166,-52.7813,66.54,92,75,14.97,CA,1722764685
2,2,paulden,34.8856,-112.4682,73.17,65,12,3.00,US,1722764687
3,3,yellowknife,62.4560,-114.3525,60.42,77,75,5.75,CA,1722764419
4,4,wailua homesteads,22.0669,-159.3780,77.79,89,20,10.36,US,1722764689


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
map_plot_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 700,
    size = "Humidity",
    color = "City"
)

# Display the map plot
map_plot_city

C:\Users\manas\anaconda3\anaconda\envs\pythondata\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df
# Zero cloudiness:
cloud_df = city_data_df[city_data_df["Cloudiness"] == 0]
# A max temperature lower than 27 degrees but higher than 21:
temp_df = cloud_df[(cloud_df["Max Temp"] > 70) & (cloud_df["Max Temp"] < 80)]
# Wind speed less than 4.5 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 10]

# Drop any rows with null values
wind_df = wind_df.dropna()


# Display sample data
wind_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
123,123,saint-pierre,-21.3393,55.4781,76.68,53,0,9.22,RE,1722764834
155,155,morondava,-20.2833,44.2833,77.14,39,0,9.48,MG,1722764871
195,195,vila franca do campo,37.7167,-25.4333,73.81,62,0,2.19,PT,1722764919
241,241,remire-montjoly,4.9167,-52.2667,73.44,100,0,1.14,GF,1722764981
283,283,laguna,38.4210,-121.4238,78.40,51,0,5.75,US,1722765034


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Use the copy() function to create a copy of the original DataFrame
hotel_df = wind_df.copy()

# Display the hotel_df DataFrame
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
123,123,saint-pierre,-21.3393,55.4781,76.68,53,0,9.22,RE,1722764834,
155,155,morondava,-20.2833,44.2833,77.14,39,0,9.48,MG,1722764871,
195,195,vila franca do campo,37.7167,-25.4333,73.81,62,0,2.19,PT,1722764919,
241,241,remire-montjoly,4.9167,-52.2667,73.44,100,0,1.14,GF,1722764981,
283,283,laguna,38.4210,-121.4238,78.40,51,0,5.75,US,1722765034,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Import API key
import api_keys

#Set up your base URL for the Geoapify Places API:
base_url = "https://api.geoapify.com/v2/places"
#Construct your API request URL by specifying the necessary parameters such as API key, location coordinates, radius, and type of place (e.g., "hotel"):

api_key = geoapify_key

for index,row in hotel_df.iterrows():
    #print(row)
    latitude = row['Lat']
    longitude = row['Lng']
    radius = 10000  # Search radius in meters
    place_type = "accommodation.hotel"
    
    url = f"{base_url}?lat={latitude}&lon={longitude}&categories={place_type}&radius={radius}&apiKey={api_key}"
    #Make a GET request to the constructed URL to retrieve the hotel information:
    import requests
    response = requests.get(url)
    name_address = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['features'][0]['properties']['name']
    except:    
        hotel_df.loc[index, "Hotel Name"] = 'No hotel found'
    
    print(name_address['features'][0]['properties']['name'])
    #Process the JSON response to extract and display the hotel names:

hotel_df.head()

Tropic Hotel
Hotel Sharon Menabe
Vinha d'Areia Beach Hotel
Complexe Belova
Holiday Inn Express & Suites
Thon Hotel Hammerfest
Hospedaje Novedades
WoodSpring Suites Chicago Darien
Hampton Inn & Suites
Holiday Inn Express
Hotel Mi Norte
Cristalina palace hotel
Hotel Complexo Mirante das Gamboas
Karratha International Hotel


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
123,123,saint-pierre,-21.3393,55.4781,76.68,53,0,9.22,RE,1722764834,Tropic Hotel
155,155,morondava,-20.2833,44.2833,77.14,39,0,9.48,MG,1722764871,Hotel Sharon Menabe
195,195,vila franca do campo,37.7167,-25.4333,73.81,62,0,2.19,PT,1722764919,Vinha d'Areia Beach Hotel
241,241,remire-montjoly,4.9167,-52.2667,73.44,100,0,1.14,GF,1722764981,Complexe Belova
283,283,laguna,38.4210,-121.4238,78.40,51,0,5.75,US,1722765034,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 700,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
map_plot_hotel

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)